In [7]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
from pathlib import Path

local_dir = "/home/christopher.x.ren/embeddings/ra_tea/planet_embeddings_v2"
local_paths = list(Path(local_dir).glob("*.parquet"))
mgrs_id_mapping = pd.read_parquet("gs://demeter-labs/tea/mgrs_id_mapping_tom_tiles_deduplicated.parquet")
reference_embedding_gdf = gpd.read_parquet(local_paths[0], columns=['geometry', 'id'])
v2_dataset = gpd.read_parquet("gs://demeter-labs/tea/classifier-datasets/tile_classifier_dataset_v2_java-sumatra.parquet")

KeyboardInterrupt: 

In [4]:

# Calculate UTM zone
reference_embedding_gdf['utm_zone'] = ((reference_embedding_gdf.geometry.centroid.x + 180) / 6 + 1).astype(int)

# Check if points are in southern hemisphere and adjust CRS
if (reference_embedding_gdf.geometry.centroid.y < 0).all():
    reference_embedding_gdf['utm_zone'] = reference_embedding_gdf['utm_zone'] + 30
results = []

# Process each UTM zone group
for zone, group in tqdm(
    reference_embedding_gdf.groupby('utm_zone'), desc="Processing UTM zones"):
    utm_crs = f'EPSG:327{zone:02d}'
    projected_group = group.to_crs(utm_crs)
    projected_group.geometry = projected_group.geometry.centroid
    reprojected_group = projected_group.to_crs('EPSG:4326')
    results.append(reprojected_group)

planet_centroid_gdf = gpd.GeoDataFrame(pd.concat(results))

/tmp/ipykernel_573102/1799520087.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reference_embedding_gdf['utm_zone'] = ((reference_embedding_gdf.geometry.centroid.x + 180) / 6 + 1).astype(int)
/tmp/ipykernel_573102/1799520087.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if (reference_embedding_gdf.geometry.centroid.y < 0).all():
Processing UTM zones: 100%|██████████| 3/3 [00:24<00:00,  8.12s/it]


In [6]:
v2_dataset['utm_zone'] = ((v2_dataset.geometry.centroid.x + 180) / 6 + 1).astype(int)
# Create empty GeoDataFrame to store final results
final_results = []

# Process each UTM zone
for zone, v2_group in v2_dataset.groupby('utm_zone'):
    # Get matching results subset for this zone
    results_subset = planet_centroid_gdf[planet_centroid_gdf.utm_zone == zone].copy()
    
    # Skip if no matching results
    if len(results_subset) == 0:
        continue
        
    # Determine correct UTM CRS
    if (v2_group.geometry.centroid.y < 0).all():
        utm_crs = f'EPSG:327{zone:02d}'  # Southern hemisphere
    else:
        utm_crs = f'EPSG:326{zone:02d}'  # Northern hemisphere
        
    # Project both datasets to UTM
    v2_utm = v2_group.to_crs(utm_crs)
    results_utm = results_subset.to_crs(utm_crs)
    # Perform spatial join
    joined = gpd.sjoin_nearest(v2_utm, results_utm, how='left', distance_col='distance')
    final_results.append(joined.to_crs('EPSG:4326'))

# Combine all results
final_df = pd.concat(final_results, ignore_index=True)
filtered_df = final_df[final_df['distance'] <= 160]
print(
    f"Dropped {len(final_df) - len(filtered_df)} rows ({(len(final_df) - len(filtered_df))/len(final_df)*100:.1f}%) due to distance > 160m")




/tmp/ipykernel_573102/179472772.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  v2_dataset['utm_zone'] = ((v2_dataset.geometry.centroid.x + 180) / 6 + 1).astype(int)
/tmp/ipykernel_573102/179472772.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if (v2_group.geometry.centroid.y < 0).all():
/tmp/ipykernel_573102/179472772.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if (v2_group.geometry.centroid.y < 0).all():
/tmp/ipykernel_573102/179472772.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.t

Dropped 25973 rows (12.0%) due to distance > 160m


In [11]:
filtered_df = final_df[final_df['distance'] <= 80]
filtered_df.to_parquet("gs://demeter-labs/tea/classifier-datasets/planet_tile_classifier_dataset_v2_java-sumatra_80m.parquet")

In [13]:
filtered_df['mgrs_id'] = [x.split("_")[0] for x in filtered_df['tile_id']]
filtered_df

/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,geometry,tile_id,class,label,utm_zone_left,index_right,id,utm_zone_right,distance,mgrs_id
0,POINT (98.77974 2.70397),47NMC_32_16_10_618_472,ei_neg,0,47,4866500,w0qpru1t0p8,47,64.806475,47NMC
2,POINT (98.7783 2.70397),47NMC_32_16_10_618_471,ei_neg,0,47,4866496,w0qpru0mnrx,47,64.827851,47NMC
3,POINT (98.79126 2.69528),47NMC_32_16_10_612_480,ei_neg,0,47,4989924,w0qr24971b5,47,60.475061,47NMC
4,POINT (98.7783 2.70107),47NMC_32_16_10_616_471,ei_neg,0,47,4865811,w0qprg8kqrx,47,63.145542,47NMC
5,POINT (98.79414 2.69384),47NMC_32_16_10_611_482,ei_neg,0,47,4989947,w0qr2476suh,47,59.863721,47NMC
...,...,...,...,...,...,...,...,...,...,...
215681,POINT (109.10724 -7.28916),49MBM_32_16_10_439_569,ei_pos,1,49,3217130,qqtwdc1t90f,49,75.395324,49MBM
215682,POINT (109.12181 -7.26896),49MBM_32_16_10_453_579,ei_pos,1,49,3222691,qqtwehu49xv,49,67.197124,49MBM
215688,POINT (109.11735 -7.29643),49MBM_32_16_10_434_576,ei_pos,1,49,3214995,qqtw7p8er33,49,76.036436,49MBM
215692,POINT (109.11892 -7.26895),49MBM_32_16_10_453_577,ei_pos,1,49,3222679,qqtwehcfwpu,49,65.874883,49MBM


In [1]:
from joblib import Parallel, delayed
import pandas as pd

filtered_df = pd.read_parquet("gs://demeter-labs/tea/classifier-datasets/planet_tile_classifier_dataset_v2_java-sumatra_80m.parquet")
filtered_df['mgrs_id'] = [x.split("_")[0] for x in filtered_df['tile_id']]

def process_mgrs(mgrs_id, tile_ids):
    # Read parquet file for this MGRS tile, filtering for relevant IDs
    path = f"gs://demeter-labs/tea/planet_embeddings/{mgrs_id}.parquet"
    df = pd.read_parquet(path, filters=[('id', 'in', tile_ids)])
    return df

# Get unique MGRS IDs and their corresponding tile IDs
mgrs_groups = filtered_df.groupby('mgrs_id')['id'].agg(list).to_dict()

# Process each MGRS tile in parallel
results = Parallel(n_jobs=3, verbose=10)(
    delayed(process_mgrs)(mgrs_id, tile_ids) 
    for mgrs_id, tile_ids in mgrs_groups.items()
)

# Combine results
embeddings_df = pd.concat(results, ignore_index=True)


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:   41.6s
/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:  2.1min
/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too shor

In [2]:
filtered_df

,geometry,tile_id,class,label,utm_zone_left,index_right,id,utm_zone_right,distance,mgrs_id
0,b'\x01\x01\x00\x00\x00d5\xaeF\xe7\xb1X@t\xf6.\...,47NMC_32_16_10_618_472,ei_neg,0,47,4866500,w0qpru1t0p8,47,64.806475,47NMC
2,b'\x01\x01\x00\x00\x00b\xef$\xb1\xcf\xb1X@\x9e...,47NMC_32_16_10_618_471,ei_neg,0,47,4866496,w0qpru0mnrx,47,64.827851,47NMC
3,b'\x01\x01\x00\x00\x00\xe8\x105\xf9\xa3\xb2X@\...,47NMC_32_16_10_612_480,ei_neg,0,47,4989924,w0qr24971b5,47,60.475061,47NMC
4,b'\x01\x01\x00\x00\x00\xaf\xe7X\xb3\xcf\xb1X@\...,47NMC_32_16_10_616_471,ei_neg,0,47,4865811,w0qprg8kqrx,47,63.145542,47NMC
5,b'\x01\x01\x00\x00\x00\xf4\x849%\xd3\xb2X@\xb5...,47NMC_32_16_10_611_482,ei_neg,0,47,4989947,w0qr2476suh,47,59.863721,47NMC
...,...,...,...,...,...,...,...,...,...,...
215681,b'\x01\x01\x00\x00\x00\xf6\x7fm\x0b\xddF[@\xd7...,49MBM_32_16_10_439_569,ei_pos,1,49,3217130,qqtwdc1t90f,49,75.395324,49MBM
215682,b'\x01\x01\x00\x00\x00\x0b\xc2\xe1\xc8\xcbG[@ ...,49MBM_32_16_10_453_579,ei_pos,1,49,3222691,qqtwehu49xv,49,67.197124,49MBM
215688,b'\x01\x01\x00\x00\x00\xb8Xf\xb3\x82G[@\x9c\x8...,49MBM_32_16_10_434_576,ei_pos,1,49,3214995,qqtw7p8er33,49,76.036436,49MBM
215692,b'\x01\x01\x00\x00\x00\xae\xe7\x95P\x9cG[@\xf7...,49MBM_32_16_10_453_577,ei_pos,1,49,3222679,qqtwehcfwpu,49,65.874883,49MBM


In [3]:
full_df = pd.merge(filtered_df[['id', 'mgrs_id', 'class', 'label']], embeddings_df, on='id', how='right')
full_df.to_parquet("gs://demeter-labs/tea/classifier-datasets/planet_tile_classifier_dataset_v2_java-sumatra_80m_embeddings.parquet")

In [1]:
full_df.shape

NameError: name 'full_df' is not defined

In [9]:
import pandas as pd
from joblib import Parallel, delayed

# Assume embeddings_df has columns 'id', 'time_period', and 'embedding'
embedding_len = len(embeddings_df.iloc[0]['embedding'])

def expand_group(period, group):
    """Expands a single group (time period) into multiple columns."""
    # Expand the list of embeddings into a DataFrame
    emb_expanded = pd.DataFrame(
        group['embedding'].tolist(),
        columns=[f'planet_{i}_{period}' for i in range(embedding_len)],
        index=group.index
    )
    # Combine the id with the expanded embeddings
    group_expanded = pd.concat([group[['id']].reset_index(drop=True),
                                emb_expanded.reset_index(drop=True)], axis=1)
    return group_expanded

# Group by time_period and prepare a list of (period, group) tuples
groups = list(embeddings_df.groupby("time_period"))

# Process each group in parallel
expanded_groups = Parallel(n_jobs=4, verbose=10)(
    delayed(expand_group)(period, group) for period, group in groups
)

# Start with a DataFrame of unique ids
expanded_df = embeddings_df[['id']].drop_duplicates()

# Merge each group's expanded DataFrame on 'id'
for group_df in expanded_groups:
    expanded_df = pd.merge(expanded_df, group_df, on='id', how='left')

print(expanded_df)


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.4min
/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=4)]: Done   7 out of  12 | elapsed:  1.5min remaining:  1.1min
/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=4)]: Done   9 out of  12 | elapsed:  2.1min remaining:   42.6s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  2.3min finished
